In [1]:
import ngsolve 
from ngsolve.webgui import Draw
import netgen.occ 
import netgen.geom2d 

import numpy as np

reference :

https://docu.ngsolve.org/latest/i-tutorials/index.html#advanced-topics

#mostly following this one: 

https://docu.ngsolve.org/latest/i-tutorials/unit-3.2-navierstokes/navierstokes.html

## Incompressible Toner Tu flocking model 

$$ (\partial_t + \lambda {\bf u} \cdot \nabla) {\bf u}  + \nabla p = (\alpha - \beta u^2) {\bf u}  
 + \nu \Delta {\bf u} $$
  
$$ \nabla \cdot {\bf u}  = 0$$

Parameters 
$\alpha, \beta, \lambda, \nu >0$, domain $\Omega$. 

By rescaling time and distance one parameter can be removed. 

The parameter $\lambda$ is dimensionless.

The parameter $\nu$ is like a diffusion coefficient (units distance$^2$/time).  

The parameter $\alpha$ has units of 1/time. 

The parameter $\beta$ has units of velocity$^2$/time. 
 

A characteristic velocity scale is 
$U = \sqrt{\alpha/\beta}$.  

The 
Reynolds number $Re = \frac{\lambda UL}{\nu}$ where $L$ is a lengthscale (of the domain?).

A dimensionless number known as the 
the Cahn number $Cn = l_c/L$ where $l_c  = \sqrt{\nu/\alpha}$ is apparently a scale above which fluctuations can grow. 
Note the Cahn number does not seem to be listed among common dimensionless numbers used in fluid mechanics on the relevant wiki site. 

I followed Navdeep Rana and Prasad Perlekar's 2020 paper entitled 
Coarsening in the 2D incompressible Toner-Tu equation: Signatures of turbulence


<b> An explicit Euler method </b>:
$$\partial_t u + {\bf L}(u) = 0$$
with $\bf L$ an operator on $u$. 
$$ \frac{u^{n+1} - u^n}{\Delta t }+ {\bf L}(u^{n}) = 0$$
Notice that the operator is applied to $u^{n}$. 
We now solve for $u^{n+1}$
$$ u^{n+1}  = ( I+ \Delta t {\bf L})u^n$$
This is simple and straightforward.  For the advection equation, it is unstable. 

<b> An implicit Euler method </b>:
$$ \frac{u^{n+1} - u^n}{\Delta t }+ {\bf L}(u^{n+1}) = 0$$
Notice that the operator is applied to $u^{n+1}$ not $u^n$. 
We now solve for $u^{n+1}$
$$ (I + \Delta t {\bf L} ) u^{n+1}  = u^n$$
$$ u^{n+1} = (I + \Delta t {\bf L} )^{-1} u^n$$
Here $I$ is the identity operator. 
Since we need the inverse of operator or a matrix, this technique is more difficult to apply but it tends to be more stable. 

If we want to use the difference  to update instead of replacing $u^{n+1}$ we start again with 
$$ (I + \Delta t {\bf L} ) u^{n+1}  = u^n$$
subtract $(I + \Delta t {\bf L} ) u^{n} $ from both sides giving 
$$ (I + \Delta t {\bf L} ) (u^{n+1}  - u^n) = u^n - (I + \Delta t {\bf L} ) u^n =- \Delta t {\bf L}  u^n $$
$$u^{n+1} - u^n = -(I + \Delta t {\bf L} )^{-1}\Delta t {\bf L}  u^n  $$


If the system is a finite element system, in place of the identity we have the mass operator. 

Let $$M(u,v) = \int_\Omega  dx\ u v $$ 
giving  
$$ u^{n+1} = (M + \Delta t {\bf L} )^{-1} u^n$$
or 
$$ u^{n+1} - u^n = (M + \Delta t {\bf L} )^{-1} (-\Delta t {\bf L}  u^n)$$

We define $$M_* = M + \Delta t {\bf L}$$ and the difference $\delta u = u^{n+1} - u^n $
The difference  can be written as 
$$ \delta u = M_*^{-1} (-\Delta t {\bf L}  u^n)$$

After computing the right hand side, you can add this to the field. 



### IMEX time discretization method 

<b> IMEX </b> stands for semi-implicit Euler method or IMplicit Explicit methods. 

A semi-implicit method
 uses the $n+1$ timestep in some of the operators (which is like an implicit  method), 
 the $n$ timestep in other operators (which is like an explicity method).
The time derivative $\partial_t u$ is approximated as $\frac{u^{n+1} - u^n}{\Delta t}$ which is first order in time.




This is helpful!  The Peclet number (using the grid size as lengthscale) is important!!!!!!
https://www.comsol.com/blogs/understanding-stabilization-methods

### Compressible Toner Tu 

$$ (\partial_t {\bf u} + \lambda {\bf u} \cdot \nabla) {\bf u}  + \nabla p/\rho = (\alpha - \beta u^2) {\bf u}  
 + \nu \Delta {\bf u} $$
  
$$ \partial_t \rho + \nabla \cdot ({\rho \bf u})  = 0 \qquad {\rm equivalently} \qquad 
 \partial_t \rho + ({\bf u} \cdot \nabla)\rho + \rho \nabla \cdot {\bf u}  = 0$$

With an equation of state  $p(\rho)$.  
For the  moment we could 
 assume that $\nabla p/\rho = c_s^2 \nabla \rho$ depends only on density with $c_s^2$ a free parameter
representing the sound speed. 

The  weak form is (and integrating by parts the term with the Laplacian in it)

$$ \int_\Omega dx [ (\partial_t {\bf u} + \lambda {\bf u} \cdot \nabla) {\bf u} ] \cdot  {\bf v}  +\int_\Omega c_s^2\nabla \rho \cdot{\bf  v} dx 
= \int_\Omega dx  (\alpha - \beta u^2) {\bf u}  \cdot {\bf v} 
 - \int_\Omega  \nu \nabla {\bf u}: \nabla {\bf v } $$

$$ \int_\Omega dx \partial_t \rho q + \int_\Omega \nabla \cdot ( \rho u) q dx = 0$$

To be clear in 2 dimensions
$$\nabla {\bf u} : \nabla {\bf v}  = \nabla u_x \cdot \nabla v_x + \nabla u_y \cdot \nabla v_y$$

Integrating by parts gives 
$$\int_\Omega dx \nabla \rho \cdot {\bf v}  = - \int_\Omega dx \rho \nabla \cdot {\bf v}$$
$$ \int_\Omega dx  \nabla \cdot (\rho {\bf u}) q dx =  - \int_\Omega dx \rho {\bf u} \cdot \nabla {\bf q} $$

The weak form becomes 
$$  \int_\Omega dx (\partial_t {\bf u}  +   \int_\Omega dx  \lambda ({\bf u} \cdot \nabla) {\bf u}  \cdot  {\bf v}   - \int_\Omega c_s^2 \rho \nabla \cdot{\bf  v} dx 
- \int_\Omega dx  (\alpha - \beta u^2) {\bf u}  \cdot {\bf v}  + \int_\Omega  \nu \nabla {\bf u}: \nabla {\bf v } =0$$

$$ \int_\Omega dx  \rho_t q  - \int_\Omega dx \rho\ ({\bf u} \cdot \nabla { q} )=0 $$



We can use a bilinear form 
$$a({\bf u},{\bf v})
= \nu \nabla {\bf u}: \nabla {\bf v } dx$$


A bilinear mass matrix 
$$ m(({\bf u},{\bf v}), (\rho,q)) = [{\bf u} \cdot {\bf v}  + \rho q] dx $$

The remaining terms are non-linear.

The convective terms 
\begin{align}
 c_u({\bf w},{\bf u},{\bf v}) &=  \lambda ({\bf w} \cdot \nabla) {\bf u} \cdot {\bf v}\ dx \\ 
 c_\rho({\bf w},\rho,q) &= - \rho\ {\bf w} \cdot \nabla q \  dx \end{align}

The propel term  
$$ {\rm propel}(w^2,{\bf u},{\bf v}) = -(\alpha - \beta w^2)   {\bf u} \cdot {\bf v}\ dx $$ 

A pressure term
$$ {\rm pres}( \rho ,{\bf u}, {\bf v} ) = - c_s^2 \rho \nabla  \cdot  {\bf v}  \ dx $$
 

Mixed methods we use ${\bf w} = {\bf u}^n$ from the previous time step.
 

The mixed method
\begin{align}
 \frac{1}{dt} \left( m({\bf u}^{n+1}, {\bf v}, \rho^{n+1},q) - m({\bf u}^{n}, {\bf v}, \rho^{n},q) \right) 
&+ a({\bf u}^{n+1}, {\bf v}) + {\rm pres}(\rho^n, {\bf u}^n, {\bf v}) + c_u ( {\bf u}^n , {\bf u}^n, {\bf v}) + c_\rho( {\bf u}^{n} , \rho^n, q) \\
&+ {\rm propel}(u^{2,n}, {\bf u}^n, {\bf v})  = 0\end{align}

Notice that only the Laplacian term is in the $n+1$ time step.  All the other terms are non-linear and harder to invert. 

\begin{align} (M + a\ dt) ( u^{n+1}, \rho^{n+1}) = M(u^n, \rho^n) - dt ( {\rm pres} + {\rm propel} +  c_u + c_\rho )(u^n,\rho^n)  
\end{align}
Subtract $(M + a\ dt )(u^n,\rho^n)$ from both sides.
\begin{align}
(M + a\ dt) (\delta u,\delta \rho) =  - dt (a() +  {\rm pres} + {\rm propel} +  c_u + c_\rho )(u^n,\rho^n)
\end{align}
Define 
$$ M_* = M + a$$
giving 
$$ (\delta u,\delta \rho) = - M_*^{-1} dt ( a +  {\rm pres} + {\rm propel} +  c_u + c_\rho )(u^n,\rho^n) $$

The differential form is used by the ngsolve IMEX example for solving the Navier-Stokes equation which is why 
I am showing it here. 

In [2]:
geo = netgen.geom2d.SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.5), bcs = ("wall", "outlet", "wall", "inlet"))
#geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl")
maxh_mesh=0.02
mesh = ngsolve.Mesh( geo.GenerateMesh(maxh=maxh_mesh))
Draw(mesh)
mesh.GetBoundaries()
#V = ngsolve.VectorH1(mesh, order=2, dirichlet="wall|inlet|outlet")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

('wall', 'outlet', 'wall', 'inlet')

In [3]:
# similar to the Taylor Hood discretization 

# make the finite element space from 3 H1 fields 
Vx = ngsolve.H1(mesh, order=2, dirichlet="inlet|wall|outlet")   # x velocity component 
Vy = ngsolve.H1(mesh, order=2, dirichlet="inlet|wall|outlet")   # y velocity component 
Q = ngsolve.H1(mesh, order=1)           # for the density field 
X = ngsolve.FESpace([Vx,Vy,Q])  # the finite element space 

# required parameters of the model
nu = 0.02    # viscosity 
cs2 = 1.0    # sound speed squared  
rho_0 = 1.0  # initial mean density
llambda = 1  # for scaling convection 
alpha = 0.2  # for propel force
beta = 1.0   # for propel force 

# check the Peclet number 
U0 = ngsolve.sqrt(alpha/beta)  # this is driving or advective velocity 
print(f'U0 = {U0}')
Peclet = U0*maxh_mesh/(2*nu)
print(f'Peclet={Peclet}')  # if this is bigger than 1 then we will have numerical instability (apparently) 
# see  https://www.comsol.com/blogs/understanding-stabilization-methods

CFL_nu_dt = maxh_mesh**2/nu   # CFL condition on dt for difussion (make dt less than this)
CFL_ad_dt = maxh_mesh/U0      # CFL condition on dt for advection (aka convection)
print(f'CFL_nu_dt = {CFL_nu_dt}, CFL_ad_dt = {CFL_ad_dt}')

dt = min(CFL_nu_dt, CFL_ad_dt)/2  # dividing by 2 for good luck
print(f'dt={dt}')

U0 = 0.4472135954999579
Peclet=0.223606797749979
CFL_nu_dt = 0.02, CFL_ad_dt = 0.044721359549995794
dt=0.01


In [4]:

# construct test and trial functions   X is finite element space 
ux,uy,rho = X.TrialFunction()
vx,vy,q = X.TestFunction()

# set up solution fields which will be updated as we evolve the PDE system
gfu = ngsolve.GridFunction(X)
velx, vely, rho_gfu = gfu.components

# compute divergences of velocities for trial and test velocities 
div_u = ngsolve.grad(ux)[0]+ngsolve.grad(uy)[1]
div_v = ngsolve.grad(vx)[0]+ngsolve.grad(vy)[1]

# viscous (laplacian) term, depends on nu (viscosity)
a = ngsolve.BilinearForm(X) 
mlap = nu*(ngsolve.grad(ux)*ngsolve.grad(vx)+ngsolve.grad(uy)*ngsolve.grad(vy) )*ngsolve.dx   #LHS, minus laplacian
a += mlap   # LHS = left hand side so I can check signs
a.Assemble()  # create the matrix operator on the finite element space 

# mstar matrix for the implicit part of the IMEX scheme: depends on dt (timestep)
mstar = ngsolve.BilinearForm(X)
mstar += (ux*vx + uy*vy + rho*q)*ngsolve.dx + dt*mlap  # notice rho q term!  laplacian consistent with LHS
mstar.Assemble()
inv = mstar.mat.Inverse(X.FreeDofs())  # construct inverse of mstar 

# set up pressure term, depends on cs2  (which is like the sound speed squared)
pterm = ngsolve.BilinearForm(X, nonassemble = True) # don't assemble because nonlinear , LHS consistent 
#RHS:-nabla Pv to P nabla v then move to LHS giving -
pterm += -cs2*div_v*rho*ngsolve.dx   

# set up convection terms, depends on llambda  
conv = ngsolve.BilinearForm(X, nonassemble = True)   # don't assemble because nonlinear
#conv += llambda*(ngsolve.Grad(u) * u) * v * ngsolve.dx LHS
conv += llambda * (ux*ngsolve.grad(ux)[0] + uy*ngsolve.grad(ux)[1]) * vx * ngsolve.dx  # for u
conv += llambda * (ux*ngsolve.grad(uy)[0] + uy*ngsolve.grad(uy)[1]) * vy * ngsolve.dx
conv += -(ux*ngsolve.grad(q)[0] + uy*ngsolve.grad(q)[1])*rho * ngsolve.dx  # for rho, because using grad rho, LHS consistent

# set up propel force term, depends on alpha,beta
propel = ngsolve.BilinearForm(X, nonassemble = True)  # don't assemble because nonlinear
#propel += (alpha - beta*ngsolve.InnerProduct(u,u)) *ngsolve.InnerProduct(u,v)*ngsolve.dx
propel += -(alpha - beta*(ux*ux + uy*uy)) * (ux*vx + uy*vy) * ngsolve.dx # LHS needs a minus sign!


In [5]:
# initial conditions, make a little vortex with a stream function  
# using a stream function so that initial condition is divergence free
x0=0.5; y0=0.25; sig0=0.07; sig2= sig0*sig0
r2 = (ngsolve.x -x0) * (ngsolve.x  - x0) + (ngsolve.y-y0) * (ngsolve.y -y0)  # is an ngsolve Coefficient function
psi_max = 0.1  # a maximum stream function value 
psi_stream = psi_max*ngsolve.exp(-0.5*r2/sig2)  # stream function  (is an ngsolve Coefficient function)

# add another little vortex 
x0=1.5; y0=0.22; sig0=0.07; sig2= sig0*sig0
r2 = (ngsolve.x -x0) * (ngsolve.x  - x0) + (ngsolve.y-y0) * (ngsolve.y -y0)
psi_max = -0.05  # a maximum stream function value 
psi_stream += psi_max*ngsolve.exp(-0.5*r2/sig2)  # add to the stream function 
# construct the velocity field from the derivatives of the stream function 
psi_x = psi_stream.Diff(ngsolve.x)  # derivative w.r.t x 
psi_y = psi_stream.Diff(ngsolve.y)

# set the initial velocity components (velx, vely are in gfu solution fields)
# I am assuming that Dirichlet boundary is not touched with this?  true!
#see https://forum.ngsolve.org/t/specifying-inconsisten-initial-and-boundary-conditions/1727
velx.Set(psi_y)
vely.Set(-1*psi_x)

# make a small density bump in the initial conditions too!
x0=0.8; y0=0.3; sig0=0.07; sig2= sig0*sig0
r2 = (ngsolve.x -x0) * (ngsolve.x  - x0) + (ngsolve.y-y0) * (ngsolve.y -y0)
rho_bump = 0.1*ngsolve.exp(-0.5*r2/sig2) + rho_0 # is a coefficient function 

# set the initial density field in the gfu solution field 
rho_gfu.Set(rho_bump)

# draw!
sceneux = Draw(velx,mesh,"u")  # these will be updated as the simulation runs 
sceneuy = Draw(vely,mesh,"u")  # these will be updated as the simulation runs 
scenep = Draw(rho_gfu,mesh,"p")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [6]:
tend = 0  # end time of simulation 

# implicit Euler/explicit Euler splitting method:IMEX
tend += 5.0   # the end time of the integration 
gfut = ngsolve.GridFunction(gfu.space,multidim=0)  # place to store outputs 
gfut.AddMultiDimComponent(gfu.vec)
t = 0; cnt = 0   # initial output and counts of number of timesteps
nn = len(velx.vec.FV().NumPy()[:])  # if you use noise you need to define a length of a vector 
wnoise = 1e-4  # noise level 
adnoise=0  # set to 1 if you want some noise 
propel_by_hand = 0  # set to 1 if you propel by hand with operator split instead of IMEX
while (t < tend-0.5*dt):
    print ("\rt=", t, end="")

    conv.Assemble()   # updates form to take into account nonlinearity of convective terms 
    propel.Assemble() # ditto for propel force 
    pterm.Assemble()  # ditto for pressure term 
 
    if (propel_by_hand ==0):
        res = (a.mat + conv.mat + pterm.mat + propel.mat)* gfu.vec
    else:
        res = (a.mat + conv.mat + pterm.mat )* gfu.vec
    gfu.vec.data -= dt * inv * res    # inverse of mstar is inv

    # propel by hand with an operator split 
    if (propel_by_hand ==1):
        upassx = velx.vec.FV().NumPy()[:] 
        upassy = vely.vec.FV().NumPy()[:] 
        fac = (alpha - beta * (upassx*upassx + upassy*upassy))
        dux = dt*fac*upassx;  duy = dt*fac*upassy
        velx.vec.FV().NumPy()[:] += dux;    vely.vec.FV().NumPy()[:] += duy

    if (adnoise==1):
        velx.vec.FV().NumPy()[:] += np.random.normal(size=nn)*wnoise 
        vely.vec.FV().NumPy()[:] += np.random.normal(size=nn)*wnoise
    
    t = t + dt; cnt += 1  # update time and counts 
    # update scenes in the webgui (look above!)
    sceneux.Redraw()
    sceneuy.Redraw()
    scenep.Redraw()
    if cnt % 5 == 0:  # how often to store outputs in the big data structure 
        gfut.AddMultiDimComponent(gfu.vec)


t= 0craete bilinearformapplication
craete bilinearformapplication
craete bilinearformapplication
t= 4.989999999999938575

In [13]:

Draw (gfut.components[0], mesh, interpolate_multidim=True, animate=True,
      min=-0.2, max=0.2, autoscale=False)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene